In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
spark

In [4]:
dataRDD= spark.sparkContext.textFile("/labs/laba01/ml-100k/u.data") # user id | item id | rating | timestamp. 

In [6]:
dataRDD.take(10)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596',
 '298\t474\t4\t884182806',
 '115\t265\t2\t881171488',
 '253\t465\t5\t891628467',
 '305\t451\t3\t886324817',
 '6\t86\t3\t883603013']

In [7]:
lst_film=dataRDD.map(lambda x: x.split("\t")).filter(lambda x: x[1]=='22').groupBy(lambda data:data[2]).mapValues(list).collect()
lst_all=dataRDD.map(lambda x: x.split("\t")).groupBy(lambda data:data[2]).mapValues(list).collect()

In [8]:
def scoring(s_list):
    index=[]
    len_lst=len(s_list)
    
    for i in range(len_lst):
        index.append(s_list[i][0])
    index.sort()
    
    reviews=[]
    for i in index:
        for j in range(len_lst):
            if i==s_list[j][0]:
                reviews.append(len(s_list[j][1])) 
    return reviews

In [9]:
answer_dict: dict = {
    'hist_film': scoring(lst_film), 
    'hist_all': scoring(lst_all)
}
with open("lab01.json", "w") as outfile:
    json.dump(answer_dict, outfile)

In [10]:
spark.stop()